In [88]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
from pymystem3 import Mystem
from string import punctuation


warnings.filterwarnings("ignore")


In [ ]:

russian_stopwords = stopwords.words("russian")

more_stop_words = ['это', 'также', 'данный', 'слово', 'который']
for word in more_stop_words:
    russian_stopwords.append(word)
russian_stopwords
# with open('assets/stopwords.csv') as f:
#     f.write(str(russian_stopwords))


In [ ]:

mystem = Mystem()

# Preprocess function


def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text


### Get reports datasets

In [332]:
dataset_answered = pd.read_csv('Reports-examples.csv', error_bad_lines=False)
# dataset_answered = dataset_answered.dropna()


In [4]:
dataset_no_answers = pd.read_csv('Reports-no-answers.csv', error_bad_lines=False)
dataset_no_answers.drop([' spare column ', '№ обращения'],inplace=True, axis=1)


In [5]:

temp_df = dataset_no_answers 
# print(dataset_no_answers)

In [ ]:
# dataset_answered
# dataset_no_answers
temp_df2 = temp_df
i = 0
for report in temp_df[' текст_обращения']:    
    # print(preprocess_text(report), '\n')
    temp_df2[' текст_обращения'][i] = preprocess_text(report)
    i += 1
temp_df2.head(10)


In [ ]:
temp_df2.to_csv('clean_no_answered_dataset.csv')


### Get clear dataset

In [9]:
dataset_no_answers = pd.read_csv('clean_no_answered_dataset.csv')
# dataset_no_answers = temp_df2[' текст_обращения'].values
dataset_no_answers = dataset_no_answers.drop('Unnamed: 0', axis=1)

dataset_no_answers = dataset_no_answers[' текст_обращения'].values

# dataset_no_answers

### NMF

In [373]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 250
# no_features = 70
no_features = 21

tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)
tfidf = tfidf_vectorizer.fit_transform(dataset_no_answers)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
# tfidf_feature_names



In [11]:

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)


### LDA

In [12]:

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features, stop_words=russian_stopwords)
tf = tf_vectorizer.fit_transform(dataset_no_answers)
tf_feature_names = tf_vectorizer.get_feature_names()
# tf_feature_names



In [13]:

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.0, random_state=0).fit(tf)

In [14]:
def display_topics(model, feature_names, no_top_words):
    result_set = set()  # множество всех слов из всех сгенерированных тем
    for topic_idx, topic in enumerate(model.components_):
        s = "Topic %d: " % (topic_idx)
        s1 = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
        for word in s1:
            result_set.add(word)
            s += word
            s += ' '
        # print(s)
    return result_set


no_top_words = 40
print('NMF', display_topics(nmf, tfidf_feature_names, no_top_words))
# print('LDA', display_topics(lda, tf_feature_names, no_top_words))


NMF {'приходить', 'предоставлять', 'сдача', 'мочь', 'ждать', 'лаборатория', 'обращаться', 'свой', 'линия', 'забор', 'очень', 'пз', 'просить', 'прием', 'требовать', 'администрация', 'время', 'горячий', 'анализ', 'обращение', 'пцр', '30', 'заказ', 'запись', 'документ', 'почта', 'врач', 'недовольный', 'код', 'сотрудник', 'человек', 'жалоба', 'кровь', 'пациент', 'связь', 'считать', 'обратный', 'сказать', 'работа', 'итог', 'день', 'работать', 'сдавать', 'очередь', 'отказывать', 'результат', 'скидка', 'принимать', 'звонок', 'медсестра', 'исследование', 'претензия', 'филиал', 'сестра', 'отказываться', 'коронавирус', 'смочь', 'регистратор', 'пациентка', 'мера', 'разбираться', 'мед', 'медицинский', 'указывать', 'хотеть', 'клиент', 'антитело', 'взять', 'выполнять', 'ребенок'}


# Algorythms ()

In [316]:
# augmentation
import random
dataset_no_answers = pd.read_csv('dataset-3-labels.csv')
dataset_no_answers.to_csv('augmented-dataset.csv')

augmented_dataset = dataset_no_answers

In [333]:

for i in range(len(dataset_no_answers['label'].array)):

    unshuffled_array = dataset_no_answers[' текст_обращения'][i]
    label = dataset_no_answers['label'][i]
    shuffled_array = unshuffled_array.split()
    shuffled_array = random.sample(shuffled_array, len(shuffled_array))
    l = len(dataset_no_answers[' текст_обращения'])
    
    

    # print(shuffled_array)
    # to_append = {' текст_обращения':shuffled_array, 'label':label}
    # print(to_append['label'])

    # все для вставки ок, но не вставляется
    # augmented_dataset.append(to_append, ignore_index=True)
    # augmented_dataset.append(shuffled_array, label)

    new_shuffled_string = ''.join(str(e)+' ' for e in shuffled_array) 
    # print(new_shuffled_string)
    # print(label)   

    with open('augmented-dataset.csv', 'a', encoding = 'utf-8') as file:
        file.write(''.join("\n"))
        file.write(str(l + i))
        file.write(',')
        file.write(new_shuffled_string)
        file.write(',')
        file.write(str(label))
    # df2 = pd.DataFrame([[new_shuffled_string, label]], columns=[' текст_обращения', 'label'])
    # augmented_dataset.append(df2)
    i += 1
    


In [343]:
augmented_dataset = pd.read_csv('augmented-dataset.csv', on_bad_lines='skip')
augmented_dataset
dataset_no_answers = augmented_dataset
dataset_no_answers

,Unnamed: 0,текст_обращения,label
0,0,суть претензия мед работник находиться филиал ...,0
1,1,доводить ваш сведение горячий линия обращаться...,1
2,2,доводить ваш сведение горячий линия обращаться...,1
3,3,доводить ваш сведение горячий линия обращаться...,1
4,4,доводить ваш сведение горячий линия обращаться...,1
...,...,...,...
279,294,год приходиться сегодня ребенок очередь обраща...,3
280,295,пациентка предъявление полагать претензия родс...,0
281,296,пациентка анализ анализ герпес g анализ пациен...,1
282,297,пз итог недовольный – медсестра рассмотрение м...,1


In [395]:
import numpy as np

no_features = 40

# dataset_no_answers = pd.read_csv('nn-labeled-dataset.csv') # 4 labels
# dataset_no_answers = pd.read_csv('dataset-3-labels.csv') # 3 labels
# dataset_no_answers = dataset_no_answers.drop('Unnamed: 0', axis=1) # только для 4 меток, там лишняя колонка

tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)

# tfidf_vectorizer = TfidfVectorizer(min_df=1, stop_words=russian_stopwords)
X = tfidf_vectorizer.fit_transform(dataset_no_answers[' текст_обращения']).toarray()
y = dataset_no_answers['label']

len(X)
# dataset_no_answers

284

In [396]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=5)

In [397]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline



# clf = RandomForestClassifier(n_estimators=10000, random_state=1, max_depth=1000)                                                # 93% best
clf = GradientBoostingClassifier(max_features=no_features, n_estimators=10000, learning_rate=0.01, max_depth=100, random_state=1) # 88%
# clf = make_pipeline(StandardScaler(), SVC(kernel='sigmoid', gamma='scale'))                           # 67%
# clf = MultinomialNB() # 67%
clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0.8837209302325582

In [398]:
y_pred = clf.predict(X_test)
y_pred

array([3, 0, 1, 3, 1, 0, 1, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 1, 0, 3, 1,
       0, 0, 1, 3, 0, 0, 1, 1, 0, 3, 3, 0, 3, 3, 3, 3, 1, 3, 3, 0, 3],
      dtype=int64)

In [399]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[12  2  0]
 [ 0  5  1]
 [ 0  2 21]]
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        14
           1       0.56      0.83      0.67         6
           3       0.95      0.91      0.93        23

    accuracy                           0.88        43
   macro avg       0.84      0.87      0.84        43
weighted avg       0.91      0.88      0.89        43

0.8837209302325582


## Export model

In [400]:
# import pickle
# s = pickle.dumps(clf)
# exported_model = pickle.loads(s)
# exported_model.predict(X_test)

In [401]:
from joblib import dump, load
dump(clf, 'GradientBoostingModel.joblib')


['GradientBoostingModel.joblib']

In [419]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
from pymystem3 import Mystem
from string import punctuation


def make_predict(report_text):
    clf = load('GradientBoostingModel.joblib')  

    dataset_to_extract_features = pd.read_csv('assets/augmented-dataset.csv', on_bad_lines='skip')
    
    text_for_features = dataset_to_extract_features[' текст_обращения']
    text_for_features = text_for_features.append(pd.Series([report_text]))

    tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)

    text_to_predict = tfidf_vectorizer.fit_transform(text_for_features).toarray()

    new_text_to_predict = text_to_predict[-1]
    new_text_to_predict = [new_text_to_predict]
    # print(new_text_to_predict)

    # text_to_predict = tfidf_vectorizer.fit_transform([report_text]).toarray()
    pr = clf.predict(new_text_to_predict)
    return pr

# random, 0
# sample_text = 'претензия хотеть клиент  –  нужно срочно попадать прием терапевт ничто толком объяснять сотрудник ничто хотеть решать отправлять пациент просить решение вопрос обратный связь'
# defined, 1
# sample_text = 'доводить ваш сведение горячий линия обращаться обращаться пз 8.30 принимать анализ антитело коронавирус т 1 регистратор большой очередь клиент указывать несоблюдение социальный дистанция регистратор кашель'
# defined, 3
sample_text = 'доводить ваш сведение горячий линия обращаться пациент пациент обращаться пз краснодар примерно 13 14 час грудной ребенок сдача анализ грубый форма сказать медсестра сдавать анализ необходимо утро т момент обращение сдавать анализ ковид пациент возмущать форма отказ'
make_predict(sample_text)

array([3], dtype=int64)

In [433]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
from pymystem3 import Mystem
from string import punctuation
from joblib import dump, load


def make_predict(report_text):
    clf = load('GradientBoostingModel.joblib')  
    russian_stopwords = ['и', 'в', 'во', 'не', 'что','он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты',
     'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня' 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
     'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там',
     'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб',
     'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним',
     'здесь', 'этом', 'один', 'почти', 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'над',
 'больше',
 'тот',
 'через',
 'эти',
 'нас',
 'про',
 'всего',
 'них',
 'какая',
 'много',
 'разве',
 'три',
 'эту',
 'моя',
 'впрочем',
 'хорошо',
 'свою',
 'этой',
 'перед',
 'иногда',
 'лучше',
 'чуть',
 'том',
 'нельзя',
 'такой',
 'им',
 'более',
 'всегда',
 'конечно',
 'всю',
 'между',
 'это',
 'также',
 'данный',
 'слово',
 'который']
    no_features = 40

    dataset_to_extract_features = pd.read_csv('assets/augmented-dataset.csv', on_bad_lines='skip')

    # with open('assets/stopwords.csv', encoding = 'utf-8') as csvfile:
    #     russian_stopwords = list(csv.reader(csvfile))
    #     print(russian_stopwords)
    

    # russian_stopwords = pd.read_csv('assets/stopwords.csv')

    text_for_features = dataset_to_extract_features[' текст_обращения']
    text_for_features = text_for_features.append(pd.Series([report_text]))

    tfidf_vectorizer = TfidfVectorizer(max_features=no_features, min_df=1, stop_words=russian_stopwords)

    text_to_predict = tfidf_vectorizer.fit_transform(text_for_features).toarray()

    new_text_to_predict = text_to_predict[-1]
    new_text_to_predict = [new_text_to_predict]
    # print(new_text_to_predict)

    # text_to_predict = tfidf_vectorizer.fit_transform([report_text]).toarray()
    pr = clf.predict(new_text_to_predict)
    return pr

# random, 0
# sample_text = 'претензия хотеть клиент  –  нужно срочно попадать прием терапевт ничто толком объяснять сотрудник ничто хотеть решать отправлять пациент просить решение вопрос обратный связь'
# defined, 1
# sample_text = 'доводить ваш сведение горячий линия обращаться обращаться пз 8.30 принимать анализ антитело коронавирус т 1 регистратор большой очередь клиент указывать несоблюдение социальный дистанция регистратор кашель'
# defined, 3
sample_text = 'доводить ваш сведение горячий линия обращаться пациент пациент обращаться пз краснодар примерно 13 14 час грудной ребенок сдача анализ грубый форма сказать медсестра сдавать анализ необходимо утро т момент обращение сдавать анализ ковид пациент возмущать форма отказ'
make_predict(sample_text)

array([3], dtype=int64)